<a href="https://colab.research.google.com/github/bereikme/korwin-portable/blob/master/Android_App_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Configuratie
PROJECT_NAME = "NfcWebhookApp"
PACKAGE_DIR = "com/example/nfcwebhook"
WEBHOOK_URL = "https://hook.eu2.make.com/w357asii29tnou7aje8e0fue8jdetqgp"

# Bestandsstructuur en inhoud
files = {
    # 1. ROOT BUILD GRADLE
    f"{PROJECT_NAME}/build.gradle.kts": """
plugins {
    id("com.android.application") version "8.2.0" apply false
    id("org.jetbrains.kotlin.android") version "1.9.20" apply false
}
""",

    # 2. SETTINGS GRADLE
    f"{PROJECT_NAME}/settings.gradle.kts": f"""
pluginManagement {{
    repositories {{
        google()
        mavenCentral()
        gradlePluginPortal()
    }}
}}
dependencyResolutionManagement {{
    repositoriesMode.set(RepositoriesMode.FAIL_ON_PROJECT_REPOS)
    repositories {{
        google()
        mavenCentral()
    }}
}}
rootProject.name = "{PROJECT_NAME}"
include(":app")
""",

    # 3. APP MODULE BUILD GRADLE
    f"{PROJECT_NAME}/app/build.gradle.kts": """
plugins {
    id("com.android.application")
    id("org.jetbrains.kotlin.android")
}

android {
    namespace = "com.example.nfcwebhook"
    compileSdk = 34

    defaultConfig {
        applicationId = "com.example.nfcwebhook"
        minSdk = 24
        targetSdk = 34
        versionCode = 1
        versionName = "1.0"
    }

    buildTypes {
        release {
            isMinifyEnabled = false
            proguardFiles(getDefaultProguardFile("proguard-android-optimize.txt"), "proguard-rules.pro")
        }
    }
    compileOptions {
        sourceCompatibility = JavaVersion.VERSION_1_8
        targetCompatibility = JavaVersion.VERSION_1_8
    }
    kotlinOptions {
        jvmTarget = "1.8"
    }
}

dependencies {
    implementation("androidx.core:core-ktx:1.12.0")
    implementation("androidx.appcompat:appcompat:1.6.1")
}
""",

    # 4. MANIFEST (Zorgt dat de app start bij NFC scan)
    f"{PROJECT_NAME}/app/src/main/AndroidManifest.xml": """
<?xml version="1.0" encoding="utf-8"?>
<manifest xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools">

    <uses-permission android:name="android.permission.INTERNET" />
    <uses-permission android:name="android.permission.NFC" />
    <uses-feature android:name="android.hardware.nfc" android:required="true" />

    <application
        android:allowBackup="true"
        android:dataExtractionRules="@xml/data_extraction_rules"
        android:fullBackupContent="@xml/backup_rules"
        android:icon="@android:drawable/ic_menu_send"
        android:label="NFC Webhook"
        android:theme="@android:style/Theme.NoDisplay"
        tools:targetApi="31">

        <!-- Theme.NoDisplay zorgt dat de app onzichtbaar lijkt -->

        <activity
            android:name=".MainActivity"
            android:exported="true"
            android:excludeFromRecents="true"
            android:theme="@android:style/Theme.Translucent.NoTitleBar">

            <intent-filter>
                <action android:name="android.intent.action.MAIN" />
                <category android:name="android.intent.category.LAUNCHER" />
            </intent-filter>

            <!-- Intent filter voor NFC Tags -->
            <intent-filter>
                <action android:name="android.nfc.action.NDEF_DISCOVERED"/>
                <category android:name="android.intent.category.DEFAULT"/>
                <data android:mimeType="*/*"/>
            </intent-filter>
            <intent-filter>
                <action android:name="android.nfc.action.TAG_DISCOVERED"/>
                <category android:name="android.intent.category.DEFAULT"/>
            </intent-filter>
            <intent-filter>
                <action android:name="android.nfc.action.TECH_DISCOVERED"/>
                <category android:name="android.intent.category.DEFAULT"/>
            </intent-filter>
        </activity>
    </application>
</manifest>
""",

    # 5. MAIN ACTIVITY (De logica)
    f"{PROJECT_NAME}/app/src/main/java/{PACKAGE_DIR}/MainActivity.kt": f"""
package com.example.nfcwebhook

import android.app.Activity
import android.content.Intent
import android.nfc.NfcAdapter
import android.nfc.Tag
import android.os.Bundle
import android.widget.Toast
import java.net.HttpURLConnection
import java.net.URL
import kotlin.concurrent.thread

class MainActivity : Activity() {{

    private val webhookUrl = "{WEBHOOK_URL}"

    override fun onCreate(savedInstanceState: Bundle?) {{
        super.onCreate(savedInstanceState)
        // Geen setContentView, we blijven onzichtbaar/transparant

        handleIntent(intent)
    }}

    override fun onNewIntent(intent: Intent?) {{
        super.onNewIntent(intent)
        handleIntent(intent)
    }}

    private fun handleIntent(intent: Intent?) {{
        if (intent == null) return

        if (NfcAdapter.ACTION_TAG_DISCOVERED == intent.action ||
            NfcAdapter.ACTION_TECH_DISCOVERED == intent.action ||
            NfcAdapter.ACTION_NDEF_DISCOVERED == intent.action) {{

            val tag: Tag? = intent.getParcelableExtra(NfcAdapter.EXTRA_TAG)
            tag?.let {{
                val tagId = bytesToHex(it.id)
                sendToWebhook(tagId)
            }}
        }} else {{
            // Als de app handmatig wordt geopend zonder NFC scan
            Toast.makeText(this, "Scan een NFC tag om te triggeren", Toast.LENGTH_SHORT).show()
            finish()
        }}
    }}

    private fun sendToWebhook(tagId: String) {{
        thread {{
            try {{
                val url = URL(webhookUrl)
                val conn = url.openConnection() as HttpURLConnection
                conn.requestMethod = "POST"
                conn.doOutput = true
                conn.setRequestProperty("Content-Type", "application/json")

                val jsonPayload = "{{\\"nfc_id\\": \\"$tagId\\", \\"timestamp\\": \\"${{System.currentTimeMillis()}}\\"}}"

                conn.outputStream.use {{ os ->
                    val input = jsonPayload.toByteArray(Charsets.UTF_8)
                    os.write(input, 0, input.size)
                }}

                val responseCode = conn.responseCode
                runOnUiThread {{
                    if (responseCode in 200..299) {{
                        Toast.makeText(this, "NFC Data Verzonden!", Toast.LENGTH_SHORT).show()
                    }} else {{
                        Toast.makeText(this, "Fout bij verzenden: $responseCode", Toast.LENGTH_LONG).show()
                    }}
                    finish() // Sluit de app direct na actie
                }}
            }} catch (e: Exception) {{
                e.printStackTrace()
                runOnUiThread {{
                    Toast.makeText(this, "Error: ${{e.message}}", Toast.LENGTH_LONG).show()
                    finish()
                }}
            }}
        }}
    }}

    private fun bytesToHex(bytes: ByteArray): String {{
        val sb = StringBuilder()
        for (b in bytes) {{
            sb.append(String.format("%02x", b))
        }}
        return sb.toString()
    }}
}}
""",

    # 6. GITHUB ACTIONS WORKFLOW
    f"{PROJECT_NAME}/.github/workflows/android.yml": """
name: Android Build & Release

on:
  push:
    branches: [ "main" ]
  workflow_dispatch:

jobs:
  build:
    runs-on: ubuntu-latest

    steps:
    - name: Checkout code
      uses: actions/checkout@v4

    - name: Set up JDK 17
      uses: actions/setup-java@v4
      with:
        java-version: '17'
        distribution: 'temurin'

    # Omdat we geen wrapper in de repo hebben (want we genereren bestanden),
    # gebruiken we Gradle direct via de setup-gradle actie of installeren we het.
    - name: Setup Gradle
      uses: gradle/actions/setup-gradle@v3
      with:
        gradle-version: '8.5'

    - name: Build Debug APK
      run: gradle assembleDebug

    - name: Upload APK
      uses: actions/upload-artifact@v4
      with:
        name: nfc-webhook-app
        path: app/build/outputs/apk/debug/app-debug.apk
""",

    # 7. DUMMY XML FILES (Nodig voor build)
    f"{PROJECT_NAME}/app/src/main/res/xml/backup_rules.xml": """<?xml version="1.0" encoding="utf-8"?><full-backup-content><include domain="sharedpref" path="."/></full-backup-content>""",
    f"{PROJECT_NAME}/app/src/main/res/xml/data_extraction_rules.xml": """<?xml version="1.0" encoding="utf-8"?><data-extraction-rules><cloud-backup><include domain="sharedpref" path="."/></cloud-backup><device-transfer><include domain="sharedpref" path="."/></device-transfer></data-extraction-rules>"""
}

def create_files():
    for path, content in files.items():
        # Maak mappen aan als ze niet bestaan
        dir_name = os.path.dirname(path)
        if dir_name and not os.path.exists(dir_name):
            os.makedirs(dir_name)

        # Schrijf bestand
        with open(path, "w", encoding="utf-8") as f:
            f.write(content.strip())

    print(f"Project '{PROJECT_NAME}' is succesvol gegenereerd!")
    print("Stappen:")
    print(f"1. Ga naar de map: cd {PROJECT_NAME}")
    print("2. Initialiseer git: git init")
    print("3. Voeg alles toe: git add .")
    print("4. Commit: git commit -m 'Initial commit'")
    print("5. Push naar GitHub en bekijk de Actions tab!")

if __name__ == "__main__":
    create_files()

Project 'NfcWebhookApp' is succesvol gegenereerd!
Stappen:
1. Ga naar de map: cd NfcWebhookApp
2. Initialiseer git: git init
3. Voeg alles toe: git add .
4. Commit: git commit -m 'Initial commit'
5. Push naar GitHub en bekijk de Actions tab!
